In [1]:
%load_ext autoreload
%autoreload 2

from chess import Game
from chess.pieces import *
import random
import time
import cProfile

In [9]:
import numpy as np

class Agent:
    
    def __init__(self):
        self.piece_value = {
            'Empty': 0,
            'Pawn': 1,
            'Horse': 3,
            'Bishop': 3,
            'Tower': 5,
            'Queen': 9,
            'King': 1e6,
        }
        
    def state_value(self, game):
        value = 0
        for row in game.board:
            for piece in row:
                piece_value = self.piece_value[type(piece).__name__]
                if piece.color == 'black':
                    value += piece_value
                else:
                    value -= piece_value
        return value
    
    def policy(self, game):
        max_value = - np.inf
        for move in game.moves('black'):
            root_node = game.simulate_move(*move)
            value = self.alphabeta(root_node, 4, False)
            if value >= max_value:
                best_move = move
                self._value = value
        return best_move

    def alphabeta(self, node, depth, alpha=-np.inf, beta=np.inf, maximizing_player=True):
        moves = node.moves(node.current_color)
        if depth == 0 or len(moves) == 0:
            return self.state_value(node)
        if maximizing_player:
            value = - np.inf
            for move in moves:
                new_node = node.simulate_move(*move)
                value = max(value, self.alphabeta(new_node, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return value
        else:
            value = np.inf
            for move in moves:
                new_node = node.simulate_move(*move)
                value = min(value, self.alphabeta(new_node, depth - 1, alpha, beta, True))
                beta = min(beta, value)
                if alpha >= beta:
                    break
            return value

In [16]:
import numpy as np
import copy

def tst():
    game = Game()
    for _ in range(1416):
        _ = np.copy(np.array([i for i in range(1000)]))
        
%timeit tst()

181 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
import numpy as np
import copy

def tst():
    game = Game()
    for _ in range(1416):
        _ = copy.deepcopy(np.array(['a' for i in range(1000)]))
        
%timeit tst()

172 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
import cProfile

game = Game()
agent = Agent()

cProfile.run('agent.alphabeta(game, 4)')

         8288782 function calls (7092262 primitive calls) in 4.540 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.540    4.540 <string>:1(<module>)
      893    0.033    0.000    0.035    0.000 agent.py:23(state_value)
   1417/1    0.014    0.000    4.540    4.540 agent.py:48(alphabeta)
   284492    0.040    0.000    0.040    0.000 board.py:20(board)
   332776    0.172    0.000    0.210    0.000 board.py:42(__getitem__)
     2832    0.004    0.000    0.005    0.000 board.py:48(__setitem__)
    43905    0.005    0.000    0.005    0.000 board.py:53(__len__)
     2310    0.004    0.000    0.004    0.000 board.py:63(__iter__)
1012440/1416    1.513    0.000    3.749    0.003 copy.py:132(deepcopy)
   642864    0.078    0.000    0.078    0.000 copy.py:190(_deepcopy_atomic)
    90624    0.245    0.000    0.632    0.000 copy.py:219(_deepcopy_tuple)
    90624    0.077    0.000    0.378    0.000 copy.

In [2]:
game = Game()
game.moves('black')

[((0, 1), (2, 2)),
 ((0, 1), (2, 0)),
 ((0, 6), (2, 7)),
 ((0, 6), (2, 5)),
 ((1, 0), (2, 0)),
 ((1, 0), (3, 0)),
 ((1, 1), (2, 1)),
 ((1, 1), (3, 1)),
 ((1, 2), (2, 2)),
 ((1, 2), (3, 2)),
 ((1, 3), (2, 3)),
 ((1, 3), (3, 3)),
 ((1, 4), (2, 4)),
 ((1, 4), (3, 4)),
 ((1, 5), (2, 5)),
 ((1, 5), (3, 5)),
 ((1, 6), (2, 6)),
 ((1, 6), (3, 6)),
 ((1, 7), (2, 7)),
 ((1, 7), (3, 7))]

In [3]:
print(game)

8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟
6                
5                
4                
3                
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
1 ♖ ♘ ♗ ♔ ♕ ♗ ♘ ♖
  a b c d e f g h


In [4]:
from chess import Agent

game = Game()
agent = Agent()


while not (game.is_checkmate(game.current_color) or game.is_draw()):
    print()
    print(game.current_color)
    if game.current_color == 'white':
        from_, to = random.choice(game.moves(game.current_color))
        new_game = game.simulate_move(from_, to)
        if new_game.is_check('white'):
            continue
    else:
        from_, to = agent.policy(game)
        print(f'agent move, value: {agent}..')
    game.move(from_, to)
    print(f"{from_} -> {to}")
    print(game)


black
agent move, value: {
    "((0, 1), (2, 2))": "0.0",
    "((0, 1), (2, 0))": "0.0",
    "((0, 6), (2, 7))": "0.0",
    "((0, 6), (2, 5))": "0.0",
    "((1, 0), (2, 0))": "0.0",
    "((1, 0), (3, 0))": "0.0",
    "((1, 1), (2, 1))": "0.0",
    "((1, 1), (3, 1))": "0.0",
    "((1, 2), (2, 2))": "0.0",
    "((1, 2), (3, 2))": "0.0",
    "((1, 3), (2, 3))": "0.0",
    "((1, 3), (3, 3))": "0.0",
    "((1, 4), (2, 4))": "0.0",
    "((1, 4), (3, 4))": "0.0",
    "((1, 5), (2, 5))": "0.0",
    "((1, 5), (3, 5))": "0.0",
    "((1, 6), (2, 6))": "0.0",
    "((1, 6), (3, 6))": "0.0",
    "((1, 7), (2, 7))": "0.0",
    "((1, 7), (3, 7))": "0.0"
}..
(1, 7) -> (3, 7)
8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟  
6                
5               ♟
4                
3                
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
1 ♖ ♘ ♗ ♔ ♕ ♗ ♘ ♖
  a b c d e f g h

white
(6, 2) -> (5, 2)
8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟  
6                
5               ♟
4                
3     ♙          
2 ♙ ♙   ♙ ♙ ♙ ♙ ♙
1 ♖ ♘ ♗ ♔ ♕ ♗ ♘ ♖
  

agent move, value: {
    "((0, 1), (2, 2))": "10.0",
    "((0, 1), (2, 0))": "10.0",
    "((0, 6), (2, 7))": "10.0",
    "((0, 6), (2, 5))": "10.0",
    "((1, 0), (2, 0))": "10.0",
    "((1, 0), (3, 0))": "10.0",
    "((1, 1), (2, 1))": "10.0",
    "((1, 1), (3, 1))": "10.0",
    "((1, 2), (2, 2))": "10.0",
    "((1, 2), (3, 2))": "10.0",
    "((1, 3), (2, 3))": "10.0",
    "((1, 3), (3, 3))": "10.0",
    "((1, 4), (2, 4))": "10.0",
    "((1, 4), (3, 4))": "10.0",
    "((1, 5), (2, 5))": "10.0",
    "((1, 5), (3, 5))": "10.0",
    "((1, 6), (2, 6))": "10.0",
    "((1, 6), (3, 6))": "10.0",
    "((1, 7), (2, 7))": "0.0",
    "((1, 7), (3, 7))": "0.0",
    "((0, 7), (1, 7))": "6.0",
    "((0, 7), (2, 7))": "6.0",
    "((3, 7), (4, 7))": "0.0",
    "((0, 7), (3, 7))": "6.0",
    "((4, 7), (5, 7))": "0.0",
    "((0, 7), (4, 7))": "6.0",
    "((5, 7), (6, 6))": "1.0",
    "((0, 7), (5, 7))": "9.0",
    "((6, 6), (7, 6))": "1.0",
    "((6, 6), (7, 5))": "4.0",
    "((6, 6), (7, 7))": "6.0",


ValueError: Missing king of color white